In [0]:
%%capture
# Install the latest Tensorflow version.
!pip3 install --upgrade tensorflow-gpu
# Install TF-Hub.
!pip3 install tensorflow-hub
!pip3 install seaborn

In [0]:
import pandas as pd
import os

import glob

# import requests
# for i in range(1000):
#   print(i)
#   url = 'https://musicsculptor.com/api/albums/{:02d}/download.metadata'.format(i)

#   myfile = requests.get(url)

#   open('{}.xlsx'.format(i), 'wb').write(myfile.content)

# xls = [file for file in os.listdir() if 'xls' in file]

# dfs = []

# for file in xls:
#   try:
#     print('File', file, 'ok')
#     df = pd.read_excel(file)
#     dfs.append(df)
#   except:
#     print('Skip.')

# df = pd.concat(dfs)


In [0]:
#df.to_csv('sculptor.csv')
df = pd.read_csv('sculptor.csv')
df_A = df[df['Track ID'].str.contains('A')]
df_A_desc = df_A[['Track Description (en)', 'Key Words (en)']]
df_A_desc

In [0]:





desc_music = df_A_desc['Track Description (en)'].str.split(" - ", n = 1, expand = True)[0] 
desc_content = df_A_desc['Track Description (en)'].str.split(" - ", n = 1, expand = True)[1] 
desc_cutting = desc_content.str.split(";", n = 1, expand = True)[1]
desc_content = desc_content.str.split(";", n = 1, expand = True)[0]
df_A_desc['desc_music'] = desc_music
df_A_desc['desc_content'] = desc_content
df_A_desc['desc_cutting'] = desc_cutting

In [0]:

df_A_desc

In [0]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

In [0]:
# #@title Compute a representation for each message, showing various lengths supported.
# word = "Elephant"
# sentence = "I am a sentence for which I would like to get its embedding."
# paragraph = (
#     "Universal Sentence Encoder embeddings also support short paragraphs. "
#     "There is no hard limit on how long the paragraph is. Roughly, the longer "
#     "the more 'diluted' the embedding will be.")
# messages = [word, sentence, paragraph]

# # Reduce logging output.
# logging.set_verbosity(logging.ERROR)

# message_embeddings = embed(messages)

# for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
#   print("Message: {}".format(messages[i]))
#   print("Embedding size: {}".format(len(message_embedding)))
#   message_embedding_snippet = ", ".join(
#       (str(x) for x in message_embedding[:3]))
#   print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

In [0]:

(df_A['desc_content']).fillna('')

In [0]:


embeddings_desc = embed(df_A_desc['Track Description (en)'])
embeddings_key = embed(df_A_desc['Key Words (en)'])
embeddings_desc_content = embed(df_A_desc['desc_content'].fillna(''))

In [0]:


df_A_desc['desc_emb'] = [e for e in np.array(embeddings_desc)]
df_A_desc['key_emb'] = [e for e in np.array(embeddings_key)]
df_A_desc['desc_content_emb'] = [e for e in np.array(embeddings_desc_content)]

In [0]:
df_A_desc




In [0]:
import scipy

In [0]:
from scipy.spatial import distance



dist_desc = distance.squareform(distance.pdist(np.array(embeddings_desc), 'cosine'))
dist_key = distance.squareform(distance.pdist(np.array(embeddings_key), 'cosine'))
dist_desc_cont = distance.squareform(distance.pdist(np.array(embeddings_desc_content), 'cosine'))


dist_combined = distance.squareform(distance.pdist(np.hstack((np.array(embeddings_key), np.array(embeddings_desc))), 'cosine'))

In [0]:

def get_top_k_sim_for_track(dist, track=0, top_k=10):


  return df_A.iloc[np.argsort(dist[track])]['Track ID'][1:1+top_k].values


top_k_key = [get_top_k_sim_for_track(dist_key, i) for i in range(len(df_A))]
top_k_desc = [get_top_k_sim_for_track(dist_desc, i) for i in range(len(df_A))]
top_k_desc_cont = [get_top_k_sim_for_track(dist_desc_cont, i) for i in range(len(df_A))]

top_k_comb = [get_top_k_sim_for_track(dist_combined, i) for i in range(len(df_A))]



In [0]:
df_A['top_10_sim_desc'] = top_k_desc
df_A['top_10_sim_desc_cont'] = top_k_desc_cont

df_A['top_10_sim_key'] = top_k_key
df_A['top_10_sim_comb'] = top_k_comb

In [0]:

df_out = df_A.drop(columns='Unnamed: 0')

#df_out.to_excel('sculptor_similarities.xlsx')

In [0]:
df_A_desc

In [0]:
def print_top_k(dist, track, top_k):

 return  [d for d in df_A_desc.iloc[np.argsort(dist[track])]['desc_content'][0:top_k+1]]



result = print_top_k(dist_desc_cont, 0, 10)

result

In [0]:










df_samples = pd.DataFrame(columns=['Anfrage', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])






In [0]:
for i in range(len(df_A)):

  result = print_top_k(dist_desc_cont, i, 10)
  df_samples.loc[len(df_samples)] = result




In [0]:
df_samples.to_excel('sculptor_similarities_descriptions_content.xlsx')

In [0]:
query = ['fashion']
query_embedding = np.array(embed(query))[0]




#scipy.spatial.distance.cosine(query, emb)
dist_query = [scipy.spatial.distance.euclidean(query_embedding, emb) for emb in df_A_desc.desc_content_emb]

df_A_desc['desc_content'].iloc[np.argsort(dist_query)][0:20]

In [0]:
 pd.set_option('display.max_rows', 10000)
 df_samples

In [0]:
len(df)